In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from Crypto.Cipher import AES
import random
from tqdm import tqdm
from collections import Counter

# Load MNIST without scaling
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(-1, 28, 28).astype(np.uint8)
test_images = test_images.reshape(-1, 28, 28).astype(np.uint8)

block_size = 4
num_blocks = (28 // block_size) ** 2
permutation_sequence = np.random.permutation(num_blocks)
key_pool = [AES.new(bytes([random.randint(0, 255) for _ in range(16)]), AES.MODE_ECB) for _ in range(1000)]

# Helper function to encrypt and permute images
def encrypt_permute_image(image, aes_key, permutation_sequence):
    encrypted_blocks = []
    for i in range(0, 28, block_size):
        for j in range(0, 28, block_size):
            block = image[i:i+block_size, j:j+block_size].flatten()
            encrypted_block = aes_key.encrypt(block.tobytes())
            encrypted_blocks.append(np.frombuffer(encrypted_block, dtype=np.uint8).reshape(block_size, block_size))
    # Apply permutation
    permuted_blocks = [encrypted_blocks[idx] for idx in permutation_sequence]
    # Reconstruct image
    permuted_image = np.zeros_like(image, dtype=np.uint8)
    idx = 0
    for i in range(0, 28, block_size):
        for j in range(0, 28, block_size):
            permuted_image[i:i+block_size, j:j+block_size] = permuted_blocks[idx]
            idx += 1
    return permuted_image

# Encrypt and permute test images as usual
def prepare_test_dataset(images, labels, key_pool, permutation_sequence, ratio=0.2):
    num_samples = int(len(images) * ratio)
    selected_indices = np.random.choice(len(images), num_samples, replace=False)
    selected_images = images[selected_indices]
    selected_labels = labels[selected_indices][:num_samples]  # Fixed to match number of samples
    
    encrypted_test_images = []
    for image in tqdm(selected_images):
        key = random.choice(key_pool)
        encrypted_image = encrypt_permute_image(image, key, permutation_sequence)
        encrypted_test_images.append(encrypted_image)
    return np.array(encrypted_test_images, dtype=np.uint8), selected_labels

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
def generate_train_test_sets(train_images, train_labels, test_images, test_labels, key_pool, permutation_sequence):
    # Encryption & Permutation
    encrypted_train_images = [encrypt_permute_image(img, random.choice(key_pool[:100]), permutation_sequence) for img in tqdm(train_images)]
    
    # train dataset ready.... (encrypted_train_images, train_labels)
    train_images_final = np.concatenate([encrypted_train_images])
    train_labels_final = np.concatenate([train_labels])
    
    print('done')

    # test dataset ready
    test_images_encrypted, test_labels_encrypted = prepare_test_dataset(test_images, test_labels, key_pool[100:1000], permutation_sequence, 1.0)
    
    print('done')
    
    train_images_final=train_images_final.reshape(-1, 28, 28, 1)/255.0
    test_images_encrypted=test_images_encrypted.reshape(-1, 28, 28, 1)/255.0
    
    return train_images_final, train_labels_final, test_images_encrypted, test_labels_encrypted

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Define gradient map computation for contour extraction with 4x4 kernel
def compute_gradient_map(images):
    gx_filter = tf.constant([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=tf.float32, shape=[3, 3, 1, 1])
    gy_filter = tf.constant([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=tf.float32, shape=[3, 3, 1, 1])
    gx = tf.nn.conv2d(images, gx_filter, strides=[1, 1, 1, 1], padding='SAME')
    gy = tf.nn.conv2d(images, gy_filter, strides=[1, 1, 1, 1], padding='SAME')
    gradient_map = tf.sqrt(tf.add(tf.square(gx), tf.square(gy)))
    return gradient_map

# Define the classification model with an LSTM head
def create_cryptoeyes_model(input_shape=(28, 28, 1)):
    # First input: encrypted images
    input_images = layers.Input(shape=input_shape)
    
    # Second input: pre-computed gradient maps
    input_gradient_maps = layers.Input(shape=input_shape)
    
    # Stream for encrypted images
    conv1_enc = layers.Conv2D(32, (1, 16), padding='same')(input_images)
    conv1_enc = layers.LeakyReLU(alpha=0.01)(conv1_enc)
    gate1_enc = layers.Conv2D(32, (1, 16), activation='sigmoid', padding='same')(input_images)
    gated1_enc = layers.Multiply()([conv1_enc, gate1_enc])
    avg1_enc = layers.Average()([gated1_enc, conv1_enc])
    pool1_enc = layers.MaxPooling2D((2, 2))(avg1_enc)
    
    conv2_enc = layers.Conv2D(64, (1, 16), padding='same')(pool1_enc)
    conv2_enc = layers.LeakyReLU(alpha=0.01)(conv2_enc)
    gate2_enc = layers.Conv2D(64, (1, 16), activation='sigmoid', padding='same')(pool1_enc)
    gated2_enc = layers.Multiply()([conv2_enc, gate2_enc])
    avg2_enc = layers.Average()([gated2_enc, conv2_enc])
    pool2_enc = layers.MaxPooling2D((2, 2))(avg2_enc)
    
    flat_enc = layers.Flatten()(pool2_enc)
    
    # Stream for gradient maps
    conv1_grad = layers.Conv2D(32, (1, 16), padding='same')(input_gradient_maps)
    conv1_grad = layers.LeakyReLU(alpha=0.01)(conv1_grad)
    gate1_grad = layers.Conv2D(32, (1, 16), activation='sigmoid', padding='same')(input_gradient_maps)
    gated1_grad = layers.Multiply()([conv1_grad, gate1_grad])
    avg1_grad = layers.Average()([gated1_grad, conv1_grad])
    pool1_grad = layers.MaxPooling2D((2, 2))(avg1_grad)
    
    conv2_grad = layers.Conv2D(64, (1, 16), padding='same')(pool1_grad)
    conv2_grad = layers.LeakyReLU(alpha=0.01)(conv2_grad)
    gate2_grad = layers.Conv2D(64, (1, 16), activation='sigmoid', padding='same')(pool1_grad)
    gated2_grad = layers.Multiply()([conv2_grad, gate2_grad])
    avg2_grad = layers.Average()([gated2_grad, conv2_grad])
    pool2_grad = layers.MaxPooling2D((2, 2))(avg2_grad)
    
    flat_grad = layers.Flatten()(pool2_grad)
    
    # Combine streams
    combined = layers.Concatenate()([flat_enc, flat_grad])
    reshaped = layers.Reshape((1, -1))(combined)  # Reshape to make it compatible with LSTM

    # Optimized LSTM activation function
    lstm_layer = layers.LSTM(128, return_sequences=False)(reshaped)
    
    output = layers.Dense(10, activation='softmax')(lstm_layer)  # Adjust the number of output classes as needed
    
    model = models.Model(inputs=[input_images, input_gradient_maps], outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.00005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [4]:
from sklearn.model_selection import train_test_split

train_images_final, train_labels_final, test_images_encrypted, test_labels_encrypted = generate_train_test_sets(train_images, train_labels, test_images, test_labels, key_pool, permutation_sequence)

input_shape = (28, 28, 1)
model = create_cryptoeyes_model(input_shape)

# Split 5% of train data for validation
train_images_train, val_images, train_labels_train, val_labels = train_test_split(
    train_images_final, train_labels_final, test_size=0.05, random_state=42
)

# Compute gradient maps for train and validation
train_gradients = compute_gradient_map(train_images_train)
val_gradients = compute_gradient_map(val_images)

# Train with validation data
model.fit(
    [train_images_train, train_gradients], train_labels_train,
    validation_data=([val_images, val_gradients], val_labels),
    epochs=20,
    batch_size=64,
    verbose=1
)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(
    [test_images_encrypted, compute_gradient_map(test_images_encrypted)], 
    test_labels_encrypted, verbose=1
)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


100%|██████████| 60000/60000 [00:17<00:00, 3347.90it/s]


done


100%|██████████| 10000/10000 [00:02<00:00, 3352.02it/s]


done


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.1988 - loss: 2.1745 - val_accuracy: 0.4900 - val_loss: 1.5318
Epoch 2/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.5176 - loss: 1.4350 - val_accuracy: 0.5750 - val_loss: 1.2652
Epoch 3/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.5846 - loss: 1.2333 - val_accuracy: 0.6083 - val_loss: 1.1484
Epoch 4/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6237 - loss: 1.1198 - val_accuracy: 0.6243 - val_loss: 1.0756
Epoch 5/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6424 - loss: 1.0504 - val_accuracy: 0.6483 - val_loss: 1.0232
Epoch 6/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6677 - loss: 0.9827 - val_accuracy: 0.6680 - val_loss: 0.9780
Epoch 7/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6814 - loss: 0.9419 - val_accuracy: 0.6797 - val_loss: 0.9396
Epoch 8/20
891/891 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6933 - loss: 0.9077 - 